# Deriving Term Frequency from CWB Weather Summary

這篇筆記要示範自然語言處理（[Natural Language Processing](https://en.wikipedia.org/wiki/Natural_language_processing), NLP）中的基本工具：計算 [term frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)。我們想要從氣象局每天的[「天氣概況」](https://www.cwb.gov.tw/V8/C/W/index.html)裡，找出氣象局專家群常常提及的一些台灣及鄰近區域的天氣現象。

以下為氣象局天氣概況的一則範例：

***
```
07fW01136
中央氣象局氣象報告
１１０年３月１３日１１時０分發布
３月１３日８時天氣概況：
一、高氣壓１０２６百帕，在北緯３２度，東經１２０度，即在江蘇，向東緩慢移動。
二、今、明（１３日、１４日）兩天東北季風影響，北部及東北部天氣較涼，其他地區早晚亦涼；臺灣東半部地區有局部短暫雨，其他地區及澎湖、金門、馬祖為多雲到晴，今日午後新竹以南山區亦有局部短暫陣雨，明日新竹以南山區亦有零星短暫雨；今、明兩天桃園至臺南、東南部（含蘭嶼、綠島）、恆春半島沿海空曠地區及澎湖、金門易有較強陣風；明日西半部地區易有局部霧或低雲影響能見度，請注意。
三、海上強風特報：
１、東北風偏強，臺灣海峽北部平均風力６至７級，最大陣風９級；臺灣北部海面、臺灣東南部海面及臺灣海峽南部平均風力可達６級，最大陣風８級，船隻請特別注意。今（１３）日臺灣東南部海面及臺灣海峽平均風力將稍減弱。明（１４日）下午起臺灣東南部海面平均風力將增強至６級，最大陣風８級，船隻請注意。
２、東北風偏強，巴士海峽、廣東海面及南海平均風力可達６級，雷雨區最大陣風８至９級，船隻請注意。今（１３日）晚起中西沙島海面平均風力將增強至６到７級，最大陣風９級，船隻請特別注意。今（１３）日廣東海面平均風力將稍減弱。

```
***

我們希望能從內容裡中找出像是「高氣壓」、「東北季風」、「局部霧」、「低雲」、以及「強風特報」、「颱風警報」這類指涉特定天氣系統或現象的詞彙，並透過分析長時間累積的文本資料，計算各個詞彙出現的頻率，來看看中央氣象局最常提到的天氣現象有哪些。

In [1]:
# 列出測試資料檔
import os, re
import pandas as pd

DATAPATH = '../data/cwb/'
SURFIX = '_1100.W01.dat'

fileinfo = []
for root, dirs, files in os.walk(DATAPATH):
    for name in files:
        date = name.replace(SURFIX, '')
        furi = (os.path.join(root, name))
        fileinfo.append({'date':date, 'uri': furi})

fileinfo = pd.DataFrame(fileinfo)
print(fileinfo)

          date                                  uri
0   2020-01-01  ../data/cwb/2020-01-01_1100.W01.dat
1   2020-02-01  ../data/cwb/2020-02-01_1100.W01.dat
2   2020-03-01  ../data/cwb/2020-03-01_1100.W01.dat
3   2020-04-01  ../data/cwb/2020-04-01_1100.W01.dat
4   2020-05-01  ../data/cwb/2020-05-01_1100.W01.dat
5   2020-06-01  ../data/cwb/2020-06-01_1100.W01.dat
6   2020-07-01  ../data/cwb/2020-07-01_1100.W01.dat
7   2020-08-01  ../data/cwb/2020-08-01_1100.W01.dat
8   2020-09-01  ../data/cwb/2020-09-01_1100.W01.dat
9   2020-10-01  ../data/cwb/2020-10-01_1100.W01.dat
10  2020-11-01  ../data/cwb/2020-11-01_1100.W01.dat
11  2020-12-01  ../data/cwb/2020-12-01_1100.W01.dat


In [7]:
# 讀取檔案
def read_cwb_summary(furi):
    import os, re
    # Read file content
    with open(furi, 'r', encoding='Big5') as f:
        text = f.readlines()
    text = ''.join(text)
    return(text)

data = read_cwb_summary(fileinfo['uri'].iloc[0])
print(data)

氣象報告
１０９年１月１日１１時０分發布
１月１日８時天氣概況：

　　高氣壓１０３６百帕，在北緯３２度，東經１２２度，即
在花鳥山海面，中心近似滯留。

　　今（１）日大陸冷氣團影響，北部及東北部天氣較冷，其
他地區早晚亦冷；明（２）日大陸冷氣團減弱，氣溫回升，早
晚仍涼；清晨新竹、苗栗沿海空曠及近山區有局部１１度左右
低溫發生的機率；今、明兩天臺灣各地及澎湖、金門、馬祖大
多為多雲到晴，僅大臺北及東半部地區有零星短暫雨。今日臺
南以北、東南部(含蘭嶼、綠島)、恆春半島及澎湖、金門、
馬祖沿海空曠地區易有８至９級強陣風，今、明兩天基隆北海
岸、東半部（含蘭嶼、綠島）及恆春半島沿海地區有長浪發生
的機率；明日清晨中南部地區有局部霧發生的機率，請注意。

　　海上強風特報：
１、東北風偏強，臺灣東南部海面及臺灣海峽北部平均風力６
至７級，最大陣風９級；臺灣北部海面及臺灣海峽南部平均風
力可達６級，最大陣風８級，船隻請特別注意。今（１）日臺
灣東南部海面及臺灣海峽北部平均風力將稍減弱。明（２）日
臺灣東南部海面及臺灣海峽北部、臺灣北部海面及臺灣海峽南
部平均風力將稍減弱。
２、東北風偏強，巴士海峽、東沙島海面及中西沙島海面平均
風力６至７級，最大陣風９級；廣東海面及南沙島海面平均風
力可達６級，最大陣風８級，船隻請特別注意。明（２）日廣
東海面及東沙島海面平均風力將稍減弱。



In [11]:
# 利用 jieba 分詞
def calculate_tf_with_jieba(article):
    import jieba
    # Perform word segmentation
    seg_list = jieba.cut(data, cut_all=False)
    # Count word frequency
    wc={}
    for w in seg_list:
        if not w in  wc:
            wc[w] = 1
        else:
            wc[w] += 1
    return(wc)

wc = calculate_tf_with_jieba(data)
print(wc)

{'氣象': 1, '報告': 1, '\n': 28, '１': 14, '０': 3, '９': 4, '年': 1, '月': 2, '日': 4, '時': 1, '分發布': 1, '８': 4, '時天氣': 1, '概況': 1, '：': 2, '\u3000': 6, '高氣壓': 1, '３': 2, '６': 5, '百帕': 1, '，': 19, '在': 2, '北緯': 1, '２': 7, '度': 3, '東經': 1, '即': 1, '花': 1, '鳥山': 1, '海面': 12, '中心': 1, '近似': 1, '滯留': 1, '。': 8, '今': 4, '（': 6, '）': 6, '日大陸': 2, '冷氣團': 2, '影響': 1, '北部': 7, '及東': 1, '天氣': 1, '較': 1, '冷': 2, '其': 1, '他': 1, '地區': 4, '早晚': 1, '亦': 1, '；': 6, '明': 3, '減弱': 4, '氣溫': 1, '回升': 1, '早': 1, '晚': 1, '仍': 1, '涼': 1, '清晨': 2, '新竹': 1, '、': 16, '苗栗': 1, '沿海': 3, '空曠及': 1, '近山區': 1, '有': 4, '局部': 2, '左右': 1, '低溫': 1, '發生': 3, '的': 3, '機率': 3, '明兩天': 2, '臺': 13, '灣': 3, '各地': 1, '及': 10, '澎湖': 2, '金門': 2, '馬': 2, '祖大': 1, '多為': 1, '多雲到': 1, '晴': 1, '僅大': 1, '北及': 1, '東': 4, '半部': 2, '零星': 1, '短': 1, '暫雨': 1, '今日': 1, '南': 2, '以北': 1, '南部': 5, '(': 1, '含蘭嶼': 2, '綠島': 2, ')': 1, '恆春半島': 1, '祖': 1, '空曠': 1, '地區易': 1, '至': 3, '級': 9, '強陣風': 1, '基隆': 1, '北海': 1, '岸': 1, '及恆春半島': 1, '有長': 1, '浪': 1, '明日'